# Proyecto I: Programación para Estadística II  

**Participantes:**  
- Anderson Martínez Abarca — *Carnet C4G876*  
- Sebastián Calvo Quesada — *Carnet C4D570*  

---

## Descripción del proyecto  
Los datos utilizados en este proyecto provienen de la **Max Planck Institute Weather Station** y corresponden a registros del clima tomados cada 10 minutos durante el año **2020**.  

Este conjunto de datos ofrece **mediciones atmosféricas completas**, incluyendo:  
- Temperatura del aire  
-  Humedad  
-  Patrones de viento  
-  Radiación  
-  Precipitación  

## Carga de datos
Para el análisis de este proyecto utilizamos los paquetes **pandas** y **os**, con el fin de gestionar la ruta de trabajo y cargar la base de datos.



In [2]:
import pandas as pd

df = pd.read_csv(r'C:\Users\ETLP\Downloads\cleaned_weather.csv')
df['date'] = pd.to_datetime(df['date'])
df.head()



,date,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,...,rho,wv,max. wv,wd,rain,raining,SWDR,PAR,max. PAR,Tlog
0,2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,...,1280.62,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45
1,2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,...,1280.33,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51
2,2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,...,1280.29,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60
3,2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,...,1281.97,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70
4,2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,...,1282.08,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81


## Registro climático
Se crea una clase con el nombre de registro climatico, la cual se encarga de mostrar la fecha, temperatura, humedad, preison y punto de rocío.

In [ ]:
#Clase RegistroClimatico
from datetime import datetime #Se utliza el paquete datatime, para manejar las fechas de cada uno de los días

class RegistroClimatico:
    def __init__(self, fecha, temperatura, humedad, presion, tdew):
        self.fecha = fecha
        self.temperatura = temperatura
        self.humedad = humedad
        self.presion = presion
        self.tdew = tdew
    
    def mostrar_info(self):
        return f"Fecha: {self.fecha}, Temperatura: {self.temperatura}°C, Humedad: {self.humedad}%, Presión: {self.presion} mbar, Punto de rocío: {self.tdew}°C"
        
    

Prueba de utilización de la clase **RegistroClimatico**.

In [ ]:
fila = df.iloc[0] #Extraemos el primer registro para la prueba
registro = RegistroClimatico(fila['date'], fila['T'], fila['rh'], fila['p'], fila['Tdew'])
registro.mostrar_info()

'Fecha: 2020-01-01 00:10:00, Temperatura: 0.71°C, Humedad: 86.1%, Presión: 1008.89 mbar, Punto de rocío: -1.33°C'

## Analizador
Se crea una clase llamada analizador, la misma se encarga de detectar registros, alertas de olas de calor, humedad extrema y puntos de rocio. Además de calcular promedios, máximos y mínimos y por último guarda los días únicos en un set.

In [6]:
class Analizador:
    def __init__(self):
        self.registros = []
        self.alertas = {}
        
    def agregar_registro(self, registro):
        self.registros.append(registro)
    
    def calcular_promedios(self):
        total_temp = total_hum = total_pres = 0
        n = len(self.registros)
        for r in self.registros:
            total_temp += r.temperatura
            total_hum += r.humedad
            total_pres += r.presion
        return {
            "Temperatura_promedio": f"{round(total_temp / n, 2)} °C",
            "Humedad_promedio": f"{round(total_hum / n, 2)} %",
            "Presion_promedio": f"{round(total_pres / n, 2)} mbar"
        }
        
    def max_min(self):
        temps = [r.temperatura for r in self.registros]
        hums = [r.humedad for r in self.registros]
        presiones = [r.presion for r in self.registros]
        return {
            "Temperatura_MAX": f"{max(temps)} °C",
            "Temperatura_MIN": f"{min(temps)} °C",
            "Humedad_MAX": f"{max(hums)} %",
            "Humedad_MIN": f"{min(hums)} %",
            "Presion_MAX": f"{max(presiones)} mbar",
            "Presion_MIN": f"{min(presiones)} mbar"
        }
    
    def generar_alerta(self, temp_umbral_calor = 40, hum_min = 20, hum_max = 90, temp_umbral_frio = 0):
        self.alertas = {"Olas_calor_detectadas": [], "Humedad_extrema_detectada": [], "Posible_rocio": [], "Ola de frío": []}
        
        for r in self.registros:
            if r.temperatura >= temp_umbral_calor:
                self.alertas["Olas_calor_detectadas"].append(r.fecha)
            if r.humedad <= hum_min or r.humedad >= hum_max:
                self.alertas["Humedad_extrema_detectada"].append(r.fecha)
            if abs(r.temperatura - r.tdew) <= 1:
                self.alertas["Posible_rocio"].append(r.fecha)
            if r.temperatura <= temp_umbral_frio:
                self.alertas["Ola de frío"].append(r.fecha)
        
        return self.alertas
    def dias_unicos(self):
    # devuelve un set de fechas (solo día)
        return {r.fecha.date() for r in self.registros}



    

Se pone a prueba el analizador en la base de datos, aplicandolo mediante un bucle for.

In [7]:
analizador = Analizador()

for i in range(len(df)):
    fila = df.iloc[i]
    registro = RegistroClimatico(fila['date'], fila['T'], fila['rh'], fila['p'], fila['Tdew'])
    analizador.agregar_registro(registro)



## Promedios
Se calculan los promedios de todos las variables. 

In [8]:
promedios = analizador.calcular_promedios()
print("Promedios de las variables\n" + "-"*28)
for k, v in promedios.items():
    print(f"+ {k.replace('_', ' ')}: {v}")



Promedios de las variables
----------------------------
+ Temperatura promedio: 10.82 °C
+ Humedad promedio: 72.49 %
+ Presion promedio: 989.99 mbar


## Máximos y mínimos 
A continuación se le presentan los máximos y mínimos de la base de datos.

In [9]:
maximos_minimos = analizador.max_min()

print("Reporte de valores extremos\n" + "-"*27)
for k, v in maximos_minimos.items():
    print(f" + {k.replace('_', ' ')}: {v}")


Reporte de valores extremos
---------------------------
 + Temperatura MAX: 34.8 °C
 + Temperatura MIN: -6.44 °C
 + Humedad MAX: 100.0 %
 + Humedad MIN: 21.16 %
 + Presion MAX: 1020.07 mbar
 + Presion MIN: 955.58 mbar


## Conteo de alertas
Se realiza un conteo de las alertas ante posibles olas de calor, humedad extrema y temperaturas muy frías (ola de frío).

In [10]:
alertas = analizador.generar_alerta(temp_umbral_calor=40, hum_min=20, hum_max=90, temp_umbral_frio= 0)
print(" Reporte de alertas meteorológicas\n" + "-"*34)

for tipo, fechas in alertas.items():
    conteo = len(fechas)
    print(f"\n + {tipo.replace('_', ' ')} : {conteo} ")
    



 Reporte de alertas meteorológicas
----------------------------------

 + Olas calor detectadas : 0 

 + Humedad extrema detectada : 11340 

 + Posible rocio : 8136 

 + Ola de frío : 2283 


In [14]:
dias = analizador.dias_unicos()
print("Total de días únicos:", len(dias))

Total de días únicos: 367


Observe que el total de días únicos nos dío como resultado 367, esto nos indica que se tomaron datos todos los días del año 2020.